<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Neural Network Framework (Keras)

## *Data Science Unit 4 Sprint 2 Assignment 3*

## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

In [1]:
import tensorflow as tf

from tensorflow.keras.datasets import boston_housing
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.1.0


In [2]:
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

In [3]:
# Before normalization
print(X_train[0])
print(X_test[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]
[ 18.0846   0.      18.1      0.       0.679    6.434  100.       1.8347
  24.     666.      20.2     27.25    29.05  ]


In [4]:
from sklearn.preprocessing import Normalizer

# Normalize the data
transformer = Normalizer().fit(X_train)
X_train = transformer.transform(X_train)
X_test = transformer.fit_transform(X_test)

# After normalization
print(X_train[0])
print(X_test[0])

[0.0024119  0.         0.01592969 0.         0.00105285 0.01201967
 0.17945359 0.00778265 0.00782786 0.6007879  0.04109624 0.77671895
 0.03663436]
[0.02675675 0.         0.02677953 0.         0.0010046  0.00951931
 0.14795322 0.0027145  0.03550877 0.98536841 0.02988655 0.04031725
 0.04298041]


In [5]:
%load_ext tensorboard

import os
import datetime

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [39]:
# Build the model

def build_model():
    model = keras.Sequential([
        layers.Dense(16, activation='relu', input_shape=[X_train.shape[1]]),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
  ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mse'])
    return model

In [40]:
model = build_model()

In [41]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 16)                224       
_________________________________________________________________
dense_16 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 17        
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Test model
example = X_train[:10]
example_result = model.predict(example)
example_result

array([[-0.2345304 ],
       [-0.16105285],
       [-0.23521966],
       [-0.18302414],
       [-0.23184004],
       [-0.22323672],
       [-0.23203965],
       [-0.2385435 ],
       [-0.22876017],
       [-0.23932964]], dtype=float32)

In [43]:
EPOCHS = 1000

history = model.fit(
    X_train, y_train, verbose=True,
    epochs=EPOCHS, validation_data=(X_test, y_test),
    callbacks=[tensorboard_callback])

Train on 404 samples, validate on 102 samples
Epoch 1/1000
404/404 [==============================] - 1s 1ms/sample - loss: 589.3233 - mse: 589.3233 - val_loss: 614.0704 - val_mse: 614.0704
Epoch 2/1000
404/404 [==============================] - 0s 136us/sample - loss: 580.9496 - mse: 580.9496 - val_loss: 605.9972 - val_mse: 605.9973
Epoch 3/1000
404/404 [==============================] - 0s 134us/sample - loss: 573.1164 - mse: 573.1164 - val_loss: 598.0854 - val_mse: 598.0853
Epoch 4/1000
404/404 [==============================] - 0s 140us/sample - loss: 565.3992 - mse: 565.3992 - val_loss: 589.5826 - val_mse: 589.5826
Epoch 5/1000
404/404 [==============================] - 0s 146us/sample - loss: 556.5119 - mse: 556.5119 - val_loss: 580.1181 - val_mse: 580.1182
Epoch 6/1000
404/404 [==============================] - 0s 143us/sample - loss: 546.7792 - mse: 546.7792 - val_loss: 569.4597 - val_mse: 569.4597
Epoch 7/1000
404/404 [==============================] - 0s 150us/sample - loss: 

In [17]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 8349), started 0:22:07 ago. (Use '!kill 8349' to kill it.)

In [21]:
# Fit Linear Regression Model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 

lm = LinearRegression()

lm.fit(X_train, y_train)
y_true = y_test
y_pred = lm.predict(X_test)

mse = mean_squared_error(y_true, y_pred)
mse

18.165510493496434

In [93]:
# use feature engineering
import pandas as pd

# reset data
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

boston_features = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
train = pd.DataFrame(X_train, columns=boston_features)
test = pd.DataFrame(X_test, columns=boston_features)

train.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [94]:
# function for feature engineering
def add_features(data):
    data = data.copy()
    data['CRIME_IND_RAT'] = data['CRIM'] / data['INDUS']
    data['COMMUTE_COST'] = data['TAX'] * (.1*data['DIS'])
    
    return data

In [95]:
train = add_features(train)
test = add_features(test)

In [96]:
test.shape

(102, 15)

In [97]:
# Normalize
from tensorflow.keras.utils import normalize

train_norm = normalize(train).values
test_norm = normalize(test).values

In [98]:
test_norm.shape

(102, 15)

In [99]:
# Build the model

def build_model2():
    model = keras.Sequential([
        layers.Dense(16, activation='relu', input_shape=[train_norm.shape[1]]),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
  ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mse'])
    return model

In [101]:
model2 = build_model2()

EPOCHS = 2000

history = model2.fit(
    train_norm, y_train, verbose=True,
    epochs=EPOCHS, validation_data=(test_norm, y_test))

Train on 404 samples, validate on 102 samples
Epoch 1/2000
404/404 [==============================] - 1s 1ms/sample - loss: 586.1555 - mse: 586.1556 - val_loss: 609.8802 - val_mse: 609.8802
Epoch 2/2000
404/404 [==============================] - 0s 104us/sample - loss: 575.6408 - mse: 575.6408 - val_loss: 599.4815 - val_mse: 599.4815
Epoch 3/2000
404/404 [==============================] - 0s 115us/sample - loss: 565.7446 - mse: 565.7447 - val_loss: 589.3905 - val_mse: 589.3904
Epoch 4/2000
404/404 [==============================] - 0s 117us/sample - loss: 555.9813 - mse: 555.9813 - val_loss: 579.2303 - val_mse: 579.2303
Epoch 5/2000
404/404 [==============================] - 0s 105us/sample - loss: 545.7127 - mse: 545.7128 - val_loss: 568.1548 - val_mse: 568.1548
Epoch 6/2000
404/404 [==============================] - 0s 126us/sample - loss: 534.5481 - mse: 534.5481 - val_loss: 556.1336 - val_mse: 556.1336
Epoch 7/2000
404/404 [==============================] - 0s 120us/sample - loss: 

In [103]:
lm = LinearRegression()

lm.fit(train_norm, y_train)
y_true = y_test
y_pred = lm.predict(test_norm)

mse = mean_squared_error(y_true, y_pred)
mse

19.28422477961286

## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- The number of nodes in your output layer should equal the number of classes you want to predict for Fashion-MNIST.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [ ]:
##### Your Code Here #####

## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.